**Uploading and extracting Mytorch:**

upload Mytorch.zip in the first cell adn run cell one and two.

In [ ]:
# upload Mytorch.zip file here
from google.colab import files
uploaded=files.upload()

In [1]:
# extracts Mytorch
import zipfile
zip_file_name = 'Mytorch.zip'
extract_location = '/content/'

with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall(extract_location)

**Importing from Mytorch:**

In [2]:
import Mytorch
import Mytorch.Network.Network as Network #to create network of layers
from Mytorch.ActivationFunctions import Sigmoid,Tanh,ReLU,LeakyReLU,Softmax #activationlayers
from Mytorch.Layers import FCLayer,Conv,Reshape
from Mytorch.LossFunctions import BinaryCrossEntropy, CrossEntropy, MeanSquaredError
from Mytorch.Optimizers import GradientDescent, GradientDescentWithMomentum, RMSProp, Adam

In [3]:
#function to calculate accuracy
def accuracy(pred,true):
    return np.sum(true==pred,axis=0)/len(true)

**Loading MNIST handwritten digit datset from pytorch:**

We're using pytroch only to get data.


In [4]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch
import numpy as np


transform = transforms.Compose([
    transforms.Resize(28),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))

])


train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)


train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)



def to_one_hot(labels, num_classes=10):
    one_hot_labels = torch.zeros(labels.size(0), num_classes)
    for i, label in enumerate(labels):
        one_hot_labels[i][label] = 1.0
    return one_hot_labels

100%|██████████| 9912422/9912422 [00:00<00:00, 108912209.60it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 103534781.05it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 29060809.30it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 20440481.51it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



**Creating network:**

network archetecture:




```
convlayer(kernelsize=3,input_channels=1,output_channels=5)
sigmoid_activation_layer()
convlayer(kernelsize=3,input_channels=5,output_channels=10)
sigmoid_activation_layer()
reshape_layer(input_shape=(10,24,24),output_shape=(10*24*24 , 1) )
fullyconnectedlayer(input_size=10*24*24 ,output_size=100)
sigmoid_activation_layer()
fullyconnectedlayer(input_size=100,output_size=10)
sigmoid_activation_layer()
```
The Reshape Layer in MyTorch is designed to transform the output from Conv(CNN) layers into a flattened shape, suitable for passing the data to Fully Connected (FCLayer) layers.

In [5]:
net=Network()
net.add_layers(Conv(kernel_size=3,input_depth=1,output_depth=5))
net.add_layers(Sigmoid())
net.add_layers(Conv(kernel_size=3,input_depth=5,output_depth=10))
net.add_layers(Sigmoid())
net.add_layers(Reshape((10,24,24),(10*24*24,1)))
net.add_layers(FCLayer(10*24*24,100))
net.add_layers(Sigmoid())
net.add_layers(FCLayer(100,10))
net.add_layers(Sigmoid())

**Training:**

In [6]:
learning_rate= 0.001
optimizer=GradientDescent(net.layers,learning_rate=learning_rate)
error=BinaryCrossEntropy()
epochs=5
for epoch in range(epochs):
    epoch_loss=[]
    acc=[]
    for images,labels in train_loader:
        data=images.reshape(images.shape[0],1,28,28)
        targets=to_one_hot(labels)
        data=data.numpy()
        targets=targets.numpy()
        outputs=net.predict(data)
        loss=error.Loss(targets,outputs)#returns loss
        epoch_loss.append(loss)
        acc.append(accuracy(np.argmax(outputs,axis=1),np.argmax(targets,axis=1)))
        error.backward(net,targets,outputs)#computes derivatives of loss with respect to each weight in each layer
        optimizer.step()# updates weights

    print(f"epoch:{epoch+1}, loss:{np.mean(epoch_loss)} , accuracy:{np.mean(acc)*100}%")

epoch:1, loss:0.1297362714460865 , accuracy:80.28833333333333%
epoch:2, loss:0.056575114539481795 , accuracy:91.85%
epoch:3, loss:0.04424243689721972 , accuracy:93.47%
epoch:4, loss:0.03710513431695464 , accuracy:94.61333333333334%
epoch:5, loss:0.03215961753225156 , accuracy:95.37166666666667%


**Testing on test dataset:**

In [7]:
acc=[]
for image,labels in test_loader:
    data=image.reshape(image.shape[0],1,28,28)
    targets=to_one_hot(labels)
    data=data.numpy()
    targets=targets.numpy()
    outputs=net.predict(data)
    acc.append(accuracy(np.argmax(outputs,axis=1),np.argmax(targets,axis=1)))

print(f"test accuracy: {np.mean(acc)*100}")


test accuracy: 95.57125796178345
